# Extracting target contigs

In order to extract the contigs representing your target sequences (the sequences that were being captured during the sequence capture process), you need to provide a fasta file containing the reference sequences for all loci of interest. Ususally all sequences of interest should be present in the file that was used to design the RNA baits for sequence capture. If you are using some standarad RNA bait library that was not specifically designed for your organism group/project (e.g. [Ultraconserved Elements - UCEs](http://ultraconserved.org/)), you can usually find the reference library on the webpage of the developer or in the respective publication. If all else fails, you can try to extract sequences of the same loci that you captured, for organisms that are closely related to your taxa, e.g. from [NCBI GenBank](https://www.ncbi.nlm.nih.gov/genbank/).

The reference library should be in simple fasta format, containign one sequence per locus of interest. Here is an example of a reference library that was used in our example dataset of palms, extracted from [Heyduk et al., 2016](https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/biolinnean/117/1/10.1111_bij.12551/1/bij12551.pdf?Expires=1501614586&Signature=I4wDg~SKpYmSEE9cEE0rZw2XLo9zcqFhPoijZ1Qeq78nX5XcnXSPwCw~9j4QxKH16BjKfkleb~wOF5RiKdTr0PmKkUYraQfeb7mD6jPBr5RgNlDcjsLV3ezZ1VyoT-4MZ9HxnYNPE0poXJcUXor1B5YYSYGBxxW1Q~zWYqzdos2VhoCP9eK7phRocpR~hhvbJx4aJDxAQdPleOpNPpknkkmoX~dTyHr2B8C7~t5v7f4DiZVEnOz0VQ9JbPw02iY13n67Qq2VfLA3HPYkui-PGfLKY0yCFKRp2xFrnjeIBV9TKZ~vuJngcua-L6URZ8cIdwzvo9GFAVDYxWUs0DywHw__&Key-Pair-Id=APKAIUCZBIA4LVPAVW3Q):

**Citation:** *Heyduk, K., Trapnell, D. W., Barrett, C. F., & Leebens-Mack, J. (2016). Phylogenomic analyses of species relationships in the genus Sabal (Arecaceae) using targeted sequence capture. Biological Journal of the Linnean Society, 117, 106–120.*

In [3]:
%%bash
head ../../data/raw/palm_reference_sequences.fasta

>Elaeis_1007_0
TGGGAGTCGCCGGGCATTTCTGGGATCTCCTCAAGCCCTACGCCCGGAACGAGGGCGTCGACTTCCTCCGGAACAAGCGCGTCGCCATCGACCTCTCCTTCTGGATCGTCCAGCACGGGGCTGCCATCCGCAACAAAACCTCTCGCCTCCGCAATCCCCACATCCGCACCACCTTTTTCCGTACTGTCGCCTTGTTT
>Elaeis_1007_1
TCGAAGATGGGGGCGTTCCCGGTGTTCGTCGTTGACGGCGAGCCATCGCCGTTGAAGACGCAGGCAAGGATGGAGCGCTTCTTCTGCAACTCTGGTGTTGATCCTTCGACGCTGTCGAAGGCGGAGGAAGGGGAGGAGTCTCCTGTGAAACAGAGGAATCAGGCATTCACCAGATGCGTTCGGGAGTGCGTG
>Elaeis_1007_2
GAACTCCTCGAAATCCTAGGGATGCCAGTTCTAAGAGCATGTGGTGAGGCTGAAGCCCTGTGTGCACAGTTAAATAGTGAAGGCCATGTTGATGCTTGCATCACTGCCGACAGTGATGCTTTCCTGTTTGGGGCGAAATGTGTCATAAAGTGCCTTCGCTCCAATTGCAAG
>Elaeis_1007_3
GACCCATTTGAGTGCTACAACATATCAGATGTTGAAGCTGGTCTTGGTTTGAAGAGAAAACAAATGGTAGCCATTGCTCTTCTGGTCGGTAATGACCATGATTTGCATGGGGTATCTGGGTTTGGGGGTTGATACGGCTGTCCGATTTGTAAAGATGTTTAGTGAGGATGAAATTTTGGCTA
>Elaeis_1007_4
GGTTATGTGAGGTTGGTAAAGGGGTTTTCCCTTTTTCAGAGGGAAGCATCAGTTTGGCCATGGATCCCCACATGCCTATTTCAAATGAGGTTTCACCGAGTGCAAGATCTCCACACTGCTCACATTGTGGTCACCCAGGCAGCAAGAAAGCTCATCAGAAGACTGCTTGTGAATACTGT

## Find and extract all target contigs
Once you got your reference fasta files ready you are good to start with extracting the contigs of interest. For this purpose we want to create an overview over which contigs represent which reference locus in each sample. At the same time we also have to be somewhat selective and discard potential duplicates that match several loci. Let's check the function that helps you do this:

In [3]:
%%bash
source activate secapr_env
secapr find_target_contigs -h

usage: secapr find_target_contigs [-h] --contigs CONTIGS --reference REFERENCE
                                  --output OUTPUT
                                  [--min-coverage MIN_COVERAGE]
                                  [--min-identity MIN_IDENTITY]
                                  [--regex REGEX] [--keep-duplicates]

Extract the contigs that match the reference database

optional arguments:
  -h, --help            show this help message and exit
  --contigs CONTIGS     The directory containing the assembled contigs in
                        fasta format.
  --reference REFERENCE
                        The fasta-file containign the reference sequences
                        (probe-order-file).
  --output OUTPUT       The directory in which to store the extracted target
                        contigs and lastz results.
  --min-coverage MIN_COVERAGE
                        The minimum percent coverage required for a match
                        [default=80].
  --min-identity 

Before running the script, you should take a look at the fasta headers in your reference fasta file. The script will use the fasta headers to extract the locus names. By default it takes the complete fasta header as the locus name. However, in many cases there is a lot of information in the fasta headers which you may not want to keep and translate as locus names (e.g. **>RPB2_intron23 Geonoma weberbaueri RNA polymerase II second largest subunit** should preferably translate into the locus name **RPB2_intron23** and discard all the rest of the header). If your fasta sequences are named consistently you can define a [regular expression](http://www.rexegg.com/regex-quickstart.html), using the `--regex` flag in order to only use the part of the string you are interested in. You can only define a single regex for the whole fasta file, which will be applied to all fasta headers in the same way.

Further, you can choose to add the `--keep-duplicates` flag, in order to also keep contigs which span across multiple loci. These will be extracted independently for each contig thhey match and may hence be present in several copies in the FASTA file containing your extracted contigs. If this flag is used a txt file with the duplicate informaiton is being printed into the output directory.

The sensitivity of the blast algorithm (LASTZ) can be altered with the flags `--min-coverage` and `--min-identity`. High values mean conservative matching requirements, while low values will return more matches but also possibly non-orthologous sequences.

Now let's run the script with default options.

    secapr find_target_contigs --contigs ../../data/processed/contigs/ --reference ../../data/raw/palm_reference_sequences.fasta --output ../../data/processed/target_contigs --assembler abyss --regex '\w+_\d+_\d+' --keep-duplicates
    
To get a first idea of the resulting matches, you can have a look at the file `match_table.txt` in the output folder.

In [13]:
import pandas as pd
table = pd.read_csv('../../data/processed/target_contigs/match_table.txt', delimiter = '\t')
table

,exon,sample_1061,sample_1063,sample_1064,sample_1065,sample_1068,sample_1070,sample_1073,sample_1074,sample_1079,sample_1080,sample_1082,sample_1083,sample_1085,sample_1086,sample_1087,sample_1140,sample_1166
0,Elaeis_197_4,1,1,1,.,1,1,1,1,1,.,1,.,1,1,1,1,1
1,Elaeis_197_5,1,1,1,.,1,1,1,.,1,.,.,.,.,1,.,1,.
2,Elaeis_793_4,1,.,1,1,1,1,.,.,1,.,1,1,.,1,1,1,1
3,Elaeis_1986_4,.,1,.,1,1,1,1,1,1,.,.,.,1,1,.,.,.
4,Elaeis_197_1,1,1,1,1,1,1,1,1,1,1,1,.,.,1,1,.,1
5,Elaeis_197_2,1,.,1,1,1,1,1,1,1,1,1,.,1,1,1,1,.
6,Elaeis_1986_1,1,1,1,1,1,1,1,1,1,.,.,1,1,.,.,.,1
7,Elaeis_807_6,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
8,Elaeis_807_4,1,1,1,1,1,1,1,1,.,1,1,1,1,1,1,.,1
9,Elaeis_1197_10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,.,.,1


Those fields containing a '1' indicate that a unique match was extracted for the respective exon and sample. If the output reveals a very low harvest of target sequences, you can try to reduce the values for the flags `--min-coverage` and `--min-identity` in order to be more generous in the matching step. If on the other hand your output turns out to capture a lot of non-homologous sequences between the different samples (can be identified after the [alignment step](align_contigs.ipynb)), you may want to turn up the values for these flags in order to be more conservative in your search.

## 2. Extract target contigs


In [3]:
%%bash
source activate secapr_env
secapr extract_contigs -h

usage: secapr extract_contigs [-h] --contigs CONTIGS --locus-db LOCUS_DB
                              --config CONFIG --output OUTPUT
                              [--verbosity {INFO,WARN,CRITICAL}]
                              [--log-path LOG_PATH]
                              [--assembler {trinity,abyss}]
                              [--include_duplicates INCLUDE_DUPLICATES]

Copyright (c) 2010-2012, Brant C. Faircloth All rights reserved.
Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met: *
Redistributions of source code must retain the above copyright notice, this
list of conditions and the following disclaimer. * Redistributions in binary
form must reproduce the above copyright notice, this list of conditions and
the following disclaimer in the documentation and/or other materials provided
with the distribution. * Neither the name of the University of California, Los
Angeles nor the nam

Now run this function by providing the paths to the contigs file, the locus-database created in the previous step (`probe.matches.sqlite`) the config file and the output folder. The required config file was automatically generated by the function `find_target_contigs` and can be found in the output folder from the previous step. Also state again, which assembler was used to generate the contigs. If you have a lot of contigs that match several reference sequences (e.g. several exons from the same gene), it might be a good idea extracting these sequences as well. For that you can add the --include_duplicates flag, as in the example below:

    secapr extract_contigs --contigs ../../data/processed/contigs/ --locus-db ../../data/processed/target_contigs/probe.matches.sqlite --config ../../data/processed/target_contigs/config --output ../../data/processed/target_contigs/extracted_contigs.fasta --assembler abyss --include_duplicates ../../data/processed/target_contigs/duplicates.txt

Now you are ready to continue to the [alignment step](align_contigs.ipynb).

[Previous page](contig_assembly.ipynb) | [Next page](align_contigs.ipynb)